In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

use_cuda = torch.cuda.is_available()
use_cuda

True

In [2]:
torch.cuda.manual_seed(12345)
batch_size = 128

train_dataset = datasets.MNIST(
    './data',
    train=True,
    download=True,
    transform=transforms.ToTensor())

test_dataset = datasets.MNIST(
    './data',
    train=False,
    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True)

In [3]:
imgs, labels = next(iter(train_loader))
print(labels[0])
transforms.ToPILImage()(imgs[0])

4


In [4]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 512)
        self.fc21 = nn.Linear(512, 64)
        self.fc22 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 512)
        self.fc4 = nn.Linear(512, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def encode(self, x):
        
        # pass the image through a linear layer and compute hidden nodes
        h = self.relu(self.fc1(x))
        
        # pass these hidden nodes into two separate linear layers
        # to generate an approximate mean and log variance for z
        return self.fc21(h), self.fc22(h)
    
    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def decode(self, z):
        h = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h))
    
    def forward(self, x):
        
        # encode img and return z distribution mean and log variance
        mu, logvar = self.encode(x.view(-1, 784)) # isnt this flattening batch?
        
        # reparameterise to generate z
        z = self.reparameterise(mu, logvar)
        
        # decode z to create reconstruction
        return self.decode(z), mu, logvar

In [5]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))
    
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # minus 0.5 ?
    # logvar.exp() ?
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # normalise
    KLD = KLD / (batch_size * 784)
    
    return BCE + KLD

In [6]:
model = VAE().cuda()
optimiser = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
log_interval = 10

def train(epoch):
    
    model.train()
    train_loss = 0
    
    for batch_idx, (img_batch, _) in enumerate(train_loader):
        
        img_batch = Variable(img_batch).cuda()
        optimiser.zero_grad()
        
        # reconstruct images
        recon_img_batch, mu, logvar = model(img_batch)
        
        # calculate loss between reconstructed and true images
        loss = loss_fn(recon_img_batch, img_batch, mu, logvar)
        loss.backward()
        
        train_loss += loss.data[0]
        optimiser.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(img_batch), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(img_batch)))
    
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    
    model.eval()
    test_loss = 0
    
    for batch_idx, (img_batch, _) in enumerate(train_loader):
        
        img_batch = Variable(img_batch, volatile=True).cuda()
        
        # reconstruct / generate images
        recon_img_batch, mu, logvar = model(img_batch)
        
        loss = loss_fn(recon_img_batch, img_batch, mu, logvar)
        test_loss += loss.data[0]
        
        # output reconstructions on every first batch (once per epoch)
        if batch_idx == 0:
            
            # take only 8 or less from batch
            n = min(img_batch.size(0), 8)
            
            # put original images and reconstructed beside each other
            orig_imgs = img_batch[:n]
            recon_imgs = recon_img_batch.view(batch_size, 1, 28, 28)[:n]
            comparison = torch.cat([orig_imgs, recon_imgs])
            
            # output comparisons to file
            save_image(comparison.data.cpu(),
                       'results/reconstruction_' + str(epoch) + '.png', nrow=n)
    
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
epochs = 100
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    
    # generate 64 latent vector samples, each a vector of size 64
    z_sample = Variable(torch.randn(64, 64)).cuda()
    
    # use these latent vectors to generate images
    sample = model.decode(z_sample).cpu()
    
    # save grid of 64 images as png
    filename = 'results/sample_' + str(epoch) + '.png'
    save_image(sample.data.view(64, 1, 28, 28), filename)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005453
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.002798
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002186
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002157
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002119
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002056
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.002056
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.001918
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001928
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001927
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001805
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001811
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001738
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001663
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001705
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001633
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001691
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001643
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001602
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001614
Train Epoch: 1 [

Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001129
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.001125
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001119
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.001146
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.001112
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.001150
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.001109
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.001085
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001084
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.001110
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.001110
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.001110
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.001102
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.001085
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.001114
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.001130
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.001124
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.001105
Train Epoch: 4 [52480/60000 (87%)]	Loss: 0.001142
Train Epoch: 4 [53760/60000 (90%)]	Loss: 0.001122


Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.001054
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.001073
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.001010
====> Epoch: 7 Average loss: 0.0011
====> Test set loss: 0.0058
Train Epoch: 8 [0/60000 (0%)]	Loss: 0.001008
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.001054
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.001091
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.001078
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.001043
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.001073
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001048
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.001017
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.001057
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.001044
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.001083
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.001031
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.001072
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.001085
Train Epoch: 8 [17920/60000 (30%)]	Loss: 0.001081
Train Epoch: 8 [19200/60000 (32%)]	Loss: 0.001074
Tr

Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.001041
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.001012
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.001067
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.001046
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.001048
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.001021
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.001015
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.001022
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.001051
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.001054
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.001070
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.001037
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.001059
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.001038
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.001049
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.001046
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.001038
Train Epoch: 11 [40960/60000 (68%)]	Loss: 0.001039
Train Epoch: 11 [42240/60000 (70%)]	Loss: 0.001106
Train Epoch: 11 [43520/60000 (7

Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.001056
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.001027
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.001012
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.001046
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.001037
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.001052
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.001011
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.001036
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.001034
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.001020
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.001020
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.001053
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.001057
====> Epoch: 14 Average loss: 0.0010
====> Test set loss: 0.0057
Train Epoch: 15 [0/60000 (0%)]	Loss: 0.001005
Train Epoch: 15 [1280/60000 (2%)]	Loss: 0.001042
Train Epoch: 15 [2560/60000 (4%)]	Loss: 0.001030
Train Epoch: 15 [3840/60000 (6%)]	Loss: 0.001058
Train Epoch: 15 [5120/60000 (9%)]	Loss: 0.001034
Train Epoch: 15 [6400/60000 (1

...

Train Epoch: 85 [56320/60000 (94%)]	Loss: 0.000992
Train Epoch: 85 [57600/60000 (96%)]	Loss: 0.001016
Train Epoch: 85 [58880/60000 (98%)]	Loss: 0.001003
====> Epoch: 85 Average loss: 0.0010
====> Test set loss: 0.0054
Train Epoch: 86 [0/60000 (0%)]	Loss: 0.000991
Train Epoch: 86 [1280/60000 (2%)]	Loss: 0.001001
Train Epoch: 86 [2560/60000 (4%)]	Loss: 0.001018
Train Epoch: 86 [3840/60000 (6%)]	Loss: 0.000999
Train Epoch: 86 [5120/60000 (9%)]	Loss: 0.000980
Train Epoch: 86 [6400/60000 (11%)]	Loss: 0.001010
Train Epoch: 86 [7680/60000 (13%)]	Loss: 0.000983
Train Epoch: 86 [8960/60000 (15%)]	Loss: 0.000995
Train Epoch: 86 [10240/60000 (17%)]	Loss: 0.000960
Train Epoch: 86 [11520/60000 (19%)]	Loss: 0.001005
Train Epoch: 86 [12800/60000 (21%)]	Loss: 0.000999
Train Epoch: 86 [14080/60000 (23%)]	Loss: 0.001012
Train Epoch: 86 [15360/60000 (26%)]	Loss: 0.000993
Train Epoch: 86 [16640/60000 (28%)]	Loss: 0.001008
Train Epoch: 86 [17920/60000 (30%)]	Loss: 0.001004
Train Epoch: 86 [19200/60000 (32%

Train Epoch: 89 [16640/60000 (28%)]	Loss: 0.000964
Train Epoch: 89 [17920/60000 (30%)]	Loss: 0.000998
Train Epoch: 89 [19200/60000 (32%)]	Loss: 0.001016
Train Epoch: 89 [20480/60000 (34%)]	Loss: 0.000986
Train Epoch: 89 [21760/60000 (36%)]	Loss: 0.001006
Train Epoch: 89 [23040/60000 (38%)]	Loss: 0.001005
Train Epoch: 89 [24320/60000 (41%)]	Loss: 0.000994
Train Epoch: 89 [25600/60000 (43%)]	Loss: 0.000970
Train Epoch: 89 [26880/60000 (45%)]	Loss: 0.000994
Train Epoch: 89 [28160/60000 (47%)]	Loss: 0.001010
Train Epoch: 89 [29440/60000 (49%)]	Loss: 0.001040
Train Epoch: 89 [30720/60000 (51%)]	Loss: 0.000956
Train Epoch: 89 [32000/60000 (53%)]	Loss: 0.000987
Train Epoch: 89 [33280/60000 (55%)]	Loss: 0.000981
Train Epoch: 89 [34560/60000 (58%)]	Loss: 0.001005
Train Epoch: 89 [35840/60000 (60%)]	Loss: 0.001049
Train Epoch: 89 [37120/60000 (62%)]	Loss: 0.000974
Train Epoch: 89 [38400/60000 (64%)]	Loss: 0.000981
Train Epoch: 89 [39680/60000 (66%)]	Loss: 0.000957
Train Epoch: 89 [40960/60000 (6

Train Epoch: 92 [38400/60000 (64%)]	Loss: 0.001015
Train Epoch: 92 [39680/60000 (66%)]	Loss: 0.001017
Train Epoch: 92 [40960/60000 (68%)]	Loss: 0.001040
Train Epoch: 92 [42240/60000 (70%)]	Loss: 0.001019
Train Epoch: 92 [43520/60000 (72%)]	Loss: 0.000957
Train Epoch: 92 [44800/60000 (75%)]	Loss: 0.001024
Train Epoch: 92 [46080/60000 (77%)]	Loss: 0.000982
Train Epoch: 92 [47360/60000 (79%)]	Loss: 0.001000
Train Epoch: 92 [48640/60000 (81%)]	Loss: 0.000976
Train Epoch: 92 [49920/60000 (83%)]	Loss: 0.001030
Train Epoch: 92 [51200/60000 (85%)]	Loss: 0.000995
Train Epoch: 92 [52480/60000 (87%)]	Loss: 0.000977
Train Epoch: 92 [53760/60000 (90%)]	Loss: 0.001004
Train Epoch: 92 [55040/60000 (92%)]	Loss: 0.001009
Train Epoch: 92 [56320/60000 (94%)]	Loss: 0.001009
Train Epoch: 92 [57600/60000 (96%)]	Loss: 0.000963
Train Epoch: 92 [58880/60000 (98%)]	Loss: 0.000968
====> Epoch: 92 Average loss: 0.0010
====> Test set loss: 0.0055
Train Epoch: 93 [0/60000 (0%)]	Loss: 0.001008
Train Epoch: 93 [1280/

====> Test set loss: 0.0055
Train Epoch: 96 [0/60000 (0%)]	Loss: 0.001021
Train Epoch: 96 [1280/60000 (2%)]	Loss: 0.000939
Train Epoch: 96 [2560/60000 (4%)]	Loss: 0.001028
Train Epoch: 96 [3840/60000 (6%)]	Loss: 0.001020
Train Epoch: 96 [5120/60000 (9%)]	Loss: 0.000970
Train Epoch: 96 [6400/60000 (11%)]	Loss: 0.000997
Train Epoch: 96 [7680/60000 (13%)]	Loss: 0.001042
Train Epoch: 96 [8960/60000 (15%)]	Loss: 0.001009
Train Epoch: 96 [10240/60000 (17%)]	Loss: 0.001029
Train Epoch: 96 [11520/60000 (19%)]	Loss: 0.000996
Train Epoch: 96 [12800/60000 (21%)]	Loss: 0.000963
Train Epoch: 96 [14080/60000 (23%)]	Loss: 0.001023
Train Epoch: 96 [15360/60000 (26%)]	Loss: 0.000984
Train Epoch: 96 [16640/60000 (28%)]	Loss: 0.000965
Train Epoch: 96 [17920/60000 (30%)]	Loss: 0.001000
Train Epoch: 96 [19200/60000 (32%)]	Loss: 0.001015
Train Epoch: 96 [20480/60000 (34%)]	Loss: 0.001005
Train Epoch: 96 [21760/60000 (36%)]	Loss: 0.001003
Train Epoch: 96 [23040/60000 (38%)]	Loss: 0.000976
Train Epoch: 96 [24

Train Epoch: 99 [23040/60000 (38%)]	Loss: 0.001000
Train Epoch: 99 [24320/60000 (41%)]	Loss: 0.000984
Train Epoch: 99 [25600/60000 (43%)]	Loss: 0.000992
Train Epoch: 99 [26880/60000 (45%)]	Loss: 0.001016
Train Epoch: 99 [28160/60000 (47%)]	Loss: 0.001017
Train Epoch: 99 [29440/60000 (49%)]	Loss: 0.001018
Train Epoch: 99 [30720/60000 (51%)]	Loss: 0.001021
Train Epoch: 99 [32000/60000 (53%)]	Loss: 0.001010
Train Epoch: 99 [33280/60000 (55%)]	Loss: 0.001020
Train Epoch: 99 [34560/60000 (58%)]	Loss: 0.000956
Train Epoch: 99 [35840/60000 (60%)]	Loss: 0.001003
Train Epoch: 99 [37120/60000 (62%)]	Loss: 0.001037
Train Epoch: 99 [38400/60000 (64%)]	Loss: 0.001010
Train Epoch: 99 [39680/60000 (66%)]	Loss: 0.001016
Train Epoch: 99 [40960/60000 (68%)]	Loss: 0.000958
Train Epoch: 99 [42240/60000 (70%)]	Loss: 0.000995
Train Epoch: 99 [43520/60000 (72%)]	Loss: 0.000960
Train Epoch: 99 [44800/60000 (75%)]	Loss: 0.000986
Train Epoch: 99 [46080/60000 (77%)]	Loss: 0.001019
Train Epoch: 99 [47360/60000 (7